In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [2]:
ThermoProperties = pd.read_csv('ThermoProperties.csv')

In [3]:
#yoinking my assignment 5 filter code and adding some more filtering/sorting

#https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy:~:text=versus%20a%20copy-,Warning,even%20before%20pandas%203.0%20is%20available.,-When%20setting%20values
pd.options.mode.copy_on_write = True

#much simplified function which properly handles cases like 1,3-dichloro-2-(chloromethyl)-2-methylpropane
def halocounter(row, suffix):
    name = row['name']
    count=0
    #make a list of prefixes along with their halogen counts
    #note that the only per-compound after removine alkenes and ethers is perfluoroisobutane, which has 10 fluorines.
    prefx = [['per', 10],['deca',10],['nona', 9],['octa', 8],['hepta', 7],['hexa', 6], ['penta', 5], ['tetra', 4], ['tri', 3], ['di', 2], ['', 1]]
    #join the prefixes to the suffixes
    groups = [[name[0] + suffix, name[1]] for name in prefx]
    for group in groups:
        if group[0] in name:
            #multiply halogen count by group count to get total halogens
            #add to running total of halogens
            count += group[1]*name.count(group[0])
            #remove used group name to avoid single halogen group causing double counts
            name = name.replace(group[0],'')
    return count


#if there are no halogens, tag the row for deletion (0)
#otherwise, add them up for a total halogen count
#ugly but it works
def allzero(row):
    if row['Fluorine Containing'] == row['Chlorine Containing'] == row['Bromine Containing'] == row['Iodine Containing'] == 0:
        result = 0
    else:
        result = row['Fluorine Containing'] + row['Chlorine Containing'] + row['Bromine Containing'] + row['Iodine Containing']
    return result

#tag carbon bond type
def aneeneyne(row):
    name = row['name']
    if 'yne' in name:
        result = 3
    elif 'ene' in name:
        result = 2
    elif 'en-' in name:
        result = 2
    elif 'phen' in name:
        result = 2
    elif 'benz' in name:
        result = 2
    elif 'aniline' in name:
        result = 2
    elif 'silane' in name:
        result = 0
    elif 'ether' in name:
        result = 0
    elif 'ester' in name:
        result = 0
    elif 'ane' in name:
        result = 1
    else:
        result = 0
    return result

def singleHalogenifier(row):
    if row['Fluorine Containing']==row['Total Halogens']:
        return '01: Fluorine'
    elif row['Chlorine Containing']==row['Total Halogens']:
        return '02: Chlorine'
    elif row['Bromine Containing']==row['Total Halogens']:
        return '03: Bromine' 
    elif row['Iodine Containing']==row['Total Halogens']:
        return '04: Iodine'
    else:
        return 'None'
    

#making a new dataframe with just halogenated alkanes
AllHalogens = ThermoProperties.copy()
#make new columns for each counts of each halogen as well as carbon bond type
#there is probably a more elegant way to do this
AllHalogens['Fluorine Containing'] = AllHalogens.apply(halocounter, suffix='fluoro', axis=1)
AllHalogens['Chlorine Containing'] = AllHalogens.apply(halocounter, suffix='chloro', axis=1)
AllHalogens['Bromine Containing'] = AllHalogens.apply(halocounter, suffix='bromo', axis=1)
AllHalogens['Iodine Containing'] = AllHalogens.apply(halocounter, suffix='iodo', axis=1)
AllHalogens['bonds'] = AllHalogens.apply(aneeneyne, axis=1)
#make new column for halogen-free deletion tag
AllHalogens['Total Halogens']= AllHalogens.apply(allzero, axis=1)

#select only rows where there is at least one halogen, and where there exist only single bonds
#then reset the index because it bugs me
AllHalogens = AllHalogens[AllHalogens['Total Halogens']>0]
AllHalogens = AllHalogens[AllHalogens['bonds']==1].reset_index()
#looking at first few rows everything seems to have worked


AllHalogens['Contained Halogen'] = AllHalogens.apply(singleHalogenifier, axis=1)
SingleHalogens = AllHalogens[~AllHalogens['Contained Halogen'].str.contains('None')]


#filer data rows are needed due to animation frame limitations
#https://plotly.com/python/animations/#current-animation-limitations-and-caveats
#Animations are designed to work well when each row of input is present across all animation frames, 
#and when categorical values mapped to symbol, color and facet are constant across frames.<----This is a problem for #Halogens (shape)
#Animations may be misleading or inconsistent if these constraints are not met.
def makefillerdict(species):
    num_halogens = [1, 2, 3, 4, 5, 6, 7, 8,10]
    filler = pd.DataFrame({'Total Halogens':num_halogens,
                 'molweight':[-100]*len(num_halogens),
                 'critical temperature (K)':[400]*len(num_halogens),
                 'acentric factor':[0.4]*len(num_halogens),
                 'boiling point (K)':[400]*len(num_halogens),
                 'Contained Halogen':[species]*len(num_halogens)})
    return filler
    

filler_fluorine = makefillerdict("01: Fluorine")
filler_chlorine = makefillerdict("02: Chlorine")
filler_bromine = makefillerdict("03: Bromine")
filler_iodine = makefillerdict("04: Iodine")
SingleHalogens = pd.concat([SingleHalogens, filler_fluorine, filler_chlorine, filler_bromine, filler_iodine])

#sort then strip numbering to make it pretty
SingleHalogens = SingleHalogens.sort_values(by=['Contained Halogen', 'Total Halogens']) #have to sort to get legend in the right order
#strip first four characters from the beginning of the Contained Halogen string ("#: ")
SingleHalogens['Contained Halogen'] = SingleHalogens['Contained Halogen'].str[4:]

#for some reason, custom markers can only be used with string inputs
#so I need to convert the number of halogens to a string here.

def stringifier(row):
    return str(row['Total Halogens'])

SingleHalogens['Total Halogens'] = SingleHalogens.apply(stringifier, axis=1)

In [4]:
"""
"Good" Figure: Represents data in a consistent manner across all attributes

Pleasing to the eye 

Uses colorscale which is not cyclical and does not fade out at one end or in the middle

hover data is named properly

axes and scales and colorbar are consistent across all data and grpah which is making it easier to understand

does not try to display too much data at once.

the short animation will tell how the boilling point of the compounds increase with incresing molecular weight

filtering with slider also allows easy comparison for veiwers

slight transparency shows overlapping points

can zoom, pan, etc. as needed

a small sidebar is added to see the changing number of halogens in compounds
"""


custom_markers = {'1': 'circle', '2': 'square', '3': 'diamond', '4':'cross', '5':'x', '6':'triangle-up', '7':'triangle-down', '8':'triangle-left', '10':'triangle-right'}

label_dict = {'molweight': 'Molecular Weight (g/mol)',
              'critical temperature (K)': 'Critical Temperature (K)',
              'boiling point (K)': 'Boiling Point (K)',
              'acentric factor': 'Acentric Factor',
              'name': 'Species'}

fig = px.scatter(SingleHalogens, x='molweight', y='boiling point (K)', 
                 color='critical temperature (K)', color_continuous_scale='plasma', 
                 size='acentric factor', template='plotly_dark',
                 hover_name='name', animation_frame='Contained Halogen', symbol='Total Halogens',
                 symbol_map=custom_markers, labels=label_dict,
                 title='Enhanced Properties of Homogeneous Haloalkanes')


fig.update_xaxes(range=[0, 600], title_text='Molecular Weight (g/mol)', tickfont=dict(size=12))
fig.update_yaxes(range=[150, 1000], title_text='Boiling Point (K)', tickfont=dict(size=12))

fig.update_layout(
    legend=dict(bgcolor='#da5b68'),
    coloraxis_colorbar=dict(title='Critical Temperature (K)', tickfont=dict(size=10)),
    title=dict(text='Enhanced Properties of Homogeneous Haloalkanes', font=dict(size=20)),
    showlegend=True,
    hoverlabel=dict(font=dict(size=12)),
    coloraxis_colorbar_orientation='h',
    coloraxis_colorbar_len=1.02,
    coloraxis_colorbar_x=0.5,
    coloraxis_colorbar_y=1.05,
    coloraxis_colorbar_ticks='outside',
    coloraxis_colorbar_tickfont=dict(size=10),
    coloraxis_colorbar_tickmode='array',
    coloraxis_colorbar_tickvals=[300, 400, 500, 600, 700, 800, 900],
    coloraxis_colorbar_ticktext=['300K', '400K', '500K', '600K', '700K', '800K', '900K'],
)

fig.update_traces(marker=dict(opacity=0.7, sizemode='diameter'))
fig.update_layout(transition=dict(duration= 50000))  # Adjust animation frame duration

fig.show()
fig.write_html("goodfig.html")